In [1]:
from __future__ import print_function

import tensorflow as tf
import pandas as pd

In [12]:
TRAIN_FILE = 'D:\Study\Ostfold\MachineLearning\git\data\ohenc_data_colNames_tensorflow.train';
TEST_FILE = 'D:\Study\Ostfold\MachineLearning\git\data\ohenc_data_colNames_tensorflow.test';

redundant_label = 'outcome<50K'
label_name = 'outcome>50K'

train = pd.read_table(TRAIN_FILE, sep=' ')
train.pop(redundant_label)

train_x, train_y = train, train.pop(label_name)
display(train_x.head(5))
display(train_y.head(5))

test = pd.read_table(TEST_FILE, sep=' ')
test.pop(redundant_label)

test_x, test_y = test, test.pop(label_name)

,age,workclassMissing,workclassFederal-gov,workclassLocal-gov,workclassNever-worked,workclassPrivate,workclassSelf-emp-inc,workclassSelf-emp-not-inc,workclassState-gov,workclassWithout-pay,...,native-countryPortugal,native-countryPuerto-Rico,native-countryScotland,native-countrySouth,native-countryTaiwan,native-countryThailand,native-countryTrinadad-Tobago,native-countryUnited-States,native-countryVietnam,native-countryYugoslavia
0,39,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,50,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0    0
1    0
2    0
3    0
4    0
Name: outcome>50K, dtype: int64

In [13]:
# Parameters
learning_rate = 0.0001
num_steps = 1500
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 1024 # 1st layer number of neurons
n_hidden_2 = 1024 # 2nd layer number of neurons
n_hidden_3 = 512
n_hidden_4 = 256
num_input = 108 # data input 
num_classes = 2 #  total classes (0 means outcome<=50)

In [14]:
# Define the input function for training
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(40000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset.make_one_shot_iterator().get_next()

In [15]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset.make_one_shot_iterator().get_next()

In [16]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))


In [17]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier1 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 2 classes.
    n_classes=2,
    optimizer=tf.train.AdagradOptimizer(learning_rate=0.0001),
    )

# Train the Model
classifier1.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=num_steps)
# Evaluate the model.
eval_result = classifier1.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E600E54C50>, '_num_ps_replicas': 0, '_task_id': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpxv8gby8r', '_service': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_master': '', '_save_summary_steps': 100, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmpxv8gby8r\model.ckpt.
INFO:tensorflow:loss = 661978.44, step = 1
INFO:tensorflow:global_step/sec: 74.8747
INFO:tensorflow:loss = 544806.25, step = 101 (1.351 sec)
INFO:tensorflow:global_step/sec: 140.545
INFO:tensorflow:loss = 327161.56, step = 201 (0.696 

In [24]:
# Build 2 hidden layer DNN with 10, 10 units respectively, dropout
classifier2 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 100 nodes each.
    hidden_units=[100, 100],
    # The model must choose between 2 classes.
    n_classes=2,
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0001),
    )

# Train the Model
classifier2.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=1500)
# Evaluate the model.
eval_result = classifier2.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E67709C400>, '_num_ps_replicas': 0, '_task_id': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpz2rtw1y7', '_service': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_master': '', '_save_summary_steps': 100, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmpz2rtw1y7\model.ckpt.
INFO:tensorflow:loss = 336482.7, step = 1
INFO:tensorflow:global_step/sec: 37.3971
INFO:tensorflow:loss = 81.902054, step = 101 (2.677 sec)
INFO:tensorflow:global_step/sec: 50.5886
INFO:tensorflow:loss = 77.880066, step = 201 (1.981 s

In [20]:
# Build 2 hidden layer DNN with 1024, 1024, 512, 256 units respectively.
classifier3 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[1024, 1024],
    # The model must choose between 2 classes.
    n_classes=2,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001),
    )

# Train the Model
classifier3.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=num_steps)
# Evaluate the model.
eval_result = classifier3.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E604571908>, '_num_ps_replicas': 0, '_task_id': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpg5yq9eja', '_service': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_master': '', '_save_summary_steps': 100, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmpg5yq9eja\model.ckpt.
INFO:tensorflow:loss = 75812.78, step = 1
INFO:tensorflow:global_step/sec: 18.5191
INFO:tensorflow:loss = 87190.836, step = 101 (5.414 sec)
INFO:tensorflow:global_step/sec: 18.3671
INFO:tensorflow:loss = 26979.092, step = 201 (5.442 s

In [25]:
# Build 2 hidden layer DNN with 1024, 1024, 512, 256 units respectively.
classifier4 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[1024, 1024],
    # The model must choose between 2 classes.
    n_classes=2,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001),
    dropout=0.5,
    )

# Train the Model
classifier4.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=num_steps)
# Evaluate the model.
eval_result = classifier4.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E678E505C0>, '_num_ps_replicas': 0, '_task_id': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpsrsaypv8', '_service': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_master': '', '_save_summary_steps': 100, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmpsrsaypv8\model.ckpt.
INFO:tensorflow:loss = 540060.6, step = 1
INFO:tensorflow:global_step/sec: 13.7845
INFO:tensorflow:loss = 135695.44, step = 101 (7.257 sec)
INFO:tensorflow:global_step/sec: 18.7829
INFO:tensorflow:loss = 162777.95, step = 201 (5.324 s

In [22]:
# Build 2 hidden layer DNN with 1024, 1024, 512, 256 units respectively, dropout
classifier5 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[1024, 1024, 512, 256],
    # The model must choose between 2 classes.
    n_classes=2,
    dropout=0.5,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001),
    )

# Train the Model
classifier5.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=num_steps)
# Evaluate the model.
eval_result = classifier5.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E678566F28>, '_num_ps_replicas': 0, '_task_id': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpab8cgo_p', '_service': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_master': '', '_save_summary_steps': 100, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmpab8cgo_p\model.ckpt.
INFO:tensorflow:loss = 529969.9, step = 1
INFO:tensorflow:global_step/sec: 10.5631
INFO:tensorflow:loss = 203577.47, step = 101 (9.485 sec)
INFO:tensorflow:global_step/sec: 11.0369
INFO:tensorflow:loss = 121828.49, step = 201 (9.062 s

In [26]:
# Build 2 hidden layer DNN with 1024, 1024, 512, 256 units respectively, dropout
classifier6 = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[1024, 256],
    # The model must choose between 2 classes.
    n_classes=2,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001),
    )

# Train the Model
classifier6.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=num_steps)
# Evaluate the model.
eval_result = classifier5.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E678D73048>, '_num_ps_replicas': 0, '_task_id': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpbr0kiva6', '_service': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_master': '', '_save_summary_steps': 100, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmpbr0kiva6\model.ckpt.
INFO:tensorflow:loss = 941233.0, step = 1
INFO:tensorflow:global_step/sec: 24.7525
INFO:tensorflow:loss = 1571.8522, step = 101 (4.047 sec)
INFO:tensorflow:global_step/sec: 40.3796
INFO:tensorflow:loss = 39194.438, step = 201 (2.473 s